In [21]:
import os
import codecs
import numpy as np

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'BFS',
 'DeepPath.ipynb',
 'env.py',
 'evaluate.py',
 'fact_prediction_eval.py',
 'link_prediction_eval.sh',
 'networks.py',
 'pathfinder.sh',
 'policy_agent.py',
 'sl_policy.py',
 'transE_eval.py',
 'transR_eval.py',
 'transX_eval.py',
 'utils.py']

In [3]:
os.listdir('./BFS/')

['BFS.py',
 'BFS.pyc',
 'DFS.pyc',
 'full_data.txt',
 'KB.py',
 'KB.pyc',
 'README',
 'run.py',
 '__init__.py',
 '__init__.pyc']

## fact_prediction_eval.py
```
def bfs_two
def get_features
ap:
TransE
TransR
RL
TransH
TransD
```

In [12]:
relation = 'concept_worksfor'

In [13]:
task_path = '../../NELL-995/tasks/' + relation +'/'

In [15]:
# dict
entity2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                  codecs.open(task_path+'entity2id.txt','r',encoding='utf-8') if len(line.split()) == 2])
relation2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                    codecs.open(task_path+'relation2id.txt','r',encoding='utf-8') if len(line.split()) == 2])

In [31]:
len(entity2id)

75492

In [32]:
len(relation2id)

400

In [129]:
# rel
rel = relation.replace("_", ":") # concept_athletehomestadium -> concept:athletehomestadium

In [22]:
# TransE
ent_vec_E = np.loadtxt(task_path+'entity2vec.unif')
rel_vec_E = np.loadtxt(task_path+'relation2vec.unif')
relation_vec_E = rel_vec_E[relation2id[rel],:] # TransE relation vector

In [34]:
relation2id[rel]

141

In [27]:
ent_vec_E.shape

(75492, 50)

In [30]:
rel_vec_E.shape

(400, 50)

In [28]:
relation_vec_E.shape

(50,)

In [131]:
# TransR
ent_vec_R = np.loadtxt(task_path+'entity2vec.bern')
rel_vec_R = np.loadtxt(task_path+'relation2vec.bern')
M_R = np.loadtxt(task_path+'A.bern') #投影矩阵
M_R = M_R.reshape([-1,50,50])
relation_vec_R = rel_vec_R[relation2id[rel],:]#关系向量TransR
M_R_vec = M_R[relation2id[rel],:,:]

In [125]:
np.loadtxt(task_path+'A.bern').shape

(20000, 50)

In [126]:
M_R.shape

(400, 50, 50)

In [127]:
M_R_vec.shape

(50, 50)

In [130]:
# TransH
ent_vec_H = np.loadtxt(task_path+'entity2vec.vec')
rel_vec_H = np.loadtxt(task_path+'relation2vec.vec')
M_H = np.loadtxt(task_path+'A.vec')
M_H = M_H.reshape([rel_vec_H.shape[0],-1])
relation_vec_H = d_r = np.expand_dims(rel_vec_H[relation2id[rel],:],1)
w_r = np.expand_dims(M_H[relation2id[rel],:],1)

In [128]:
M_H.shape

(400, 50)

In [138]:
# TransD
ent_vec_D = np.loadtxt(task_path+'entity2vec.vec_D')
rel_vec_D = np.loadtxt(task_path+'relation2vec.vec_D')
M_D = np.loadtxt(task_path+'A.vec_D')
ent_num = ent_vec_D.shape[0]
rel_num = rel_vec_D.shape[0]
rel_tran = M_D[0:rel_num,:]
ent_tran = M_D[rel_num:,:]
dim_D = ent_vec_D.shape[1]

r = np.expand_dims(rel_vec_D[relation2id[rel],:], 1)
r_p = np.expand_dims(rel_tran[relation2id[rel],:], 1)

In [50]:
def get_features(task_path,relation2id):
    stats = dict([(line.split('\t')[0],int(line.split('\t')[1])) for line in \
                  codecs.open(task_path+'path_stats.txt','r',encoding='utf-8') if len(line.split()) == 2])
    max_freq = max(stats.values()) #路径最大次数
    useful_paths = [] # ids path list
    named_paths = [] # rel_names path list
    paths = [line.rstrip() for line in codecs.open(task_path+'path_to_use.txt','r',encoding='utf-8')]
    for path in paths:
        # filter: not in stats and count less
        if path not in stats:
            continue
        elif max_freq > 1 and stats[path] < 2: 
            continue
        # filter: len(path)<=0
        if len(path.split(' -> ')) <= 10:
            pathIndex = []
            pathName = []
            relations = path.split(' -> ')
            for rel in relations:
                pathName.append(rel)
                rel_id = relation2id[rel]
                pathIndex.append(rel_id)
            useful_paths.append(pathIndex)
            named_paths.append(pathName)
    return useful_paths, named_paths

In [51]:
# features
id_path,name_path = get_features(task_path,relation2id)
path_weights = np.array([1.0/len(path) for path in name_path])

In [54]:
id_path

[[130], [254], [15], [193], [252], [35]]

In [53]:
name_path

[['concept:personleadsorganization'],
 ['concept:organizationhiredperson_inv'],
 ['concept:agentcollaborateswithagent_inv'],
 ['concept:journalistwritesforpublication'],
 ['concept:mutualproxyfor_inv'],
 ['concept:organizationterminatedperson_inv']]

In [55]:
path_weights

array([1., 1., 1., 1., 1., 1.])

In [61]:
class KB(object):
    def __init__(self):
        self.entities = {} # {'实体id':[Path.{relation,entity2},]}

    def addRelation(self, entity1, relation, entity2):
        # add direct connections
        if entity1 in self.entities.keys():
            self.entities[entity1].append(Path(relation, entity2))
        else:
            self.entities[entity1] = [Path(relation, entity2)] # entities{entity1：Path{.relation.connected_entity}(relation, entity2)}

    def getPathsFrom(self, entity):
        return self.entities[entity]

    def removePath(self, entity1, entity2):
        # remove direct connection between e1 and e2
        for idx, path in enumerate(self.entities[entity1]):
            if(path.connected_entity == entity2):
                del self.entities[entity1][idx]
                break
        for idx, path in enumerate(self.entities[entity2]):
            if(path.connected_entity == entity1):
                del self.entities[entity2][idx]
                break

    def pickRandomIntermediatesBetween(self, entity1, entity2, num):
        #TO DO: COULD BE IMPROVED BY NARROWING THE RANGE OF RANDOM EACH TIME ITERATIVELY CHOOSE AN INTERMEDIATE  
        from sets import Set
        import random

        res = Set()
        if num > len(self.entities) - 2:
            raise ValueError('Number of Intermediates picked is larger than possible', 'num_entities: {}'.format(len(self.entities)), 'num_itermediates: {}'.format(num))
        for i in range(num):
            itermediate = random.choice(self.entities.keys())
            while itermediate in res or itermediate == entity1 or itermediate == entity2:
                itermediate = random.choice(self.entities.keys())
            res.add(itermediate)
        return list(res)

    def __str__(self):
        string = ""
        for entity in self.entities:
            string += entity + ','.join(str(x) for x in self.entities[entity])
            string += '\n'
        return string


class Path(object):
    def __init__(self, relation, connected_entity):
        self.relation = relation
        self.connected_entity = connected_entity

    def __str__(self):
        return "\t{}\t{}".format(self.relation, self.connected_entity)

    __repr__ = __str__

In [62]:
# kb
kb = KB()       #知识库
kb_inv = KB()   #逆向
for line in codecs.open(task_path+'graph.txt'):
    e1,rel,e2 = line.split()
    kb.addRelation(e1,rel,e2)     # entities{entity1：Path{.relation.connected_entity}(relation, entity2)}
    kb_inv.addRelation(e2,rel,e1)

In [66]:
len(kb.entities)

75227

In [67]:
len(kb_inv.entities)

75227

In [65]:
kb.entities['concept_politicsblog_perspective']

[	concept:proxyfor	concept_book_new,
 	concept:locationlocatedwithinlocation_inv	concept_lake_new,
 	concept:atlocation_inv	concept_beverage_new]

In [70]:
kb_inv.entities['concept_politicsblog_perspective']

[	concept:proxyfor_inv	concept_book_new,
 	concept:locationlocatedwithinlocation	concept_lake_new,
 	concept:atlocation	concept_beverage_new]

In [146]:
# read_samples
# thing$concept_politician_sam_sullivan,thing$concept_city_whistler:-
def read_samples(task_path,pairs_file='train.pairs'):
    pairs,labels = [],[]
    for line in codecs.open(task_path+pairs_file,'r',encoding='utf-8'):
        e1 = line.split(',')[0].replace('thing$','')
        e2 = line.split(',')[1].split(':')[0].replace('thing$','')
        if (e1 not in kb.entities) or (e2 not in kb.entities):
            continue
        pairs.append((e1,e2))
        labels.append(1 if line[-2] == '+' else 0)
    return pairs,labels

In [147]:
test_pairs,test_labels = read_samples(task_path,'sort_test.pairs')

In [72]:
test_pairs[:5]

[('concept_actor_peter_king', 'concept_blog_sports_illustrated'),
 ('concept_celebrity_sam_phillips', 'concept_company_post'),
 ('concept_celebrity_sam_phillips', 'concept_company_sun'),
 ('concept_celebrity_sam_phillips', 'concept_company_sun_microsystems001'),
 ('concept_celebrity_sam_phillips', 'concept_newspaper_journal')]

In [74]:
len(test_pairs)

2698

In [73]:
test_labels[:5]

[1, 0, 1, 0, 0]

In [107]:
set().add('a')

In [120]:
def bi_path_search(e1,e2,path,kb,kb_inv):
    '''
    :the bidirectional search for reasoning
    e1,r1,ex,r2,e2
    path:start-><-end:r1,r2
    kb:->:e1,r1,ex|ex,r2,e2
    kb_inv:<-:ex,r1,e1|e2,r2,ex
    '''
    start,end = 0,len(path)
    left,right = set(),set()
    left.add(e1)
    right.add(e2)
    left_path,right_path = [],[]
    while(start < end):
        left_step = path[start]
        left_next = set()
        right_step = path[end-1]
        right_next = set()

        if len(left) < len(right):
            left_path.append(left_step)
            start += 1
            for entity in left: # BFS loop
                try:
                    for path_ in kb.getPathsFrom(entity):
                        if path_.relation == left_step:
                            left_next.add(path_.connected_entity)
                except Exception as e:
                    #print('len(left):',len(left),left,'not such entity')
                    return False
            left = left_next

        else: # start->end;kb->kb_inv
            right_path.append(right_step)
            end -= 1
            for entity in right:
                try:
                    for path_ in kb_inv.getPathsFrom(entity):
                        if path_.relation == right_step:
                            right_next.add(path_.connected_entity)
                except Exception as e:
                    #print('len(right):',len(right),right,'not such entity')
                    return False
            right = right_next
    #if len(right & left) != 0:
     #   print(left,right)
    return True if len(right & left) != 0 else False

In [178]:
# aps
def entity2vec(ent_vec,entity,M_vec=None,w_r=None):
    if isinstance(w_r,np.ndarray):
        ent = np.expand_dims(ent_vec[entity2id[entity],:],1)
        ent_ = ent - np.matmul(w_r.transpose(), ent)*w_r
        return ent_
    if isinstance(M_vec,np.ndarray):
        return np.matmul(ent_vec[entity2id[entity],:], M_vec)
    return ent_vec[entity2id[entity],:]

def l2_score(h,r,t):
    return -np.sum(np.square(h + r - t))

scores_E = [l2_score(entity2vec(ent_vec_E,pair[0]),relation_vec_E,entity2vec(ent_vec_E,pair[1])) for pair in test_pairs]
scores_R = [l2_score(entity2vec(ent_vec_R,pair[0],M_vec=M_R_vec),relation_vec_R,entity2vec(ent_vec_R,pair[1],M_vec=M_R_vec)) for pair in test_pairs]
scores_rl = [sum(path_weights*[int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path]) for pair in test_pairs]
scores_H = [l2_score(entity2vec(ent_vec_H,pair[0],w_r=w_r),relation_vec_H,entity2vec(ent_vec_H,pair[1],w_r=w_r)) for pair in test_pairs]

def score_D(ent_vec_D,ent_tran,pair):
    h = np.expand_dims(ent_vec_D[entity2id[pair[0]],:], 1)
    h_p = np.expand_dims(ent_tran[entity2id[pair[0]],:], 1)
    t = np.expand_dims(ent_vec_D[entity2id[pair[1]],:], 1)
    t_p = np.expand_dims(ent_tran[entity2id[pair[1]],:], 1)
    M_rh = np.matmul(r_p, h_p.transpose()) + np.identity(dim)
    M_rt = np.matmul(r_p, t_p.transpose()) + np.identity(dim)
    score = - np.sum(np.square(M_rh.dot(h) + r - M_rt.dot(t)))
    return score
    
scores_D =[score_D(ent_vec_D,ent_tran,pair) for pair in test_pairs] 

def ap(scores,test_labels):
    # evaluate rank quality
    # 1 1 1 0 0 0 better than 0 0 0 1 1 1
    rank_stats = sorted(zip(scores, test_labels),key = lambda x:x[0], reverse=True)
    correct = 0
    ranks = []
    for idx, item in enumerate(rank_stats):
        if item[1] == 1:
            correct += 1
            ranks.append(correct/(1.0+idx)) # append precision
    return np.mean(ranks) if len(ranks) != 0 else 0

models = {
    'E':ap(scores_E, test_labels),
    'R':ap(scores_R, test_labels),
    'rl':ap(scores_rl, test_labels),
    'H':ap(scores_H, test_labels),
    'D':ap(scores_D, test_labels)
}

In [179]:
models

{'D': 0.23670935347580135,
 'E': 0.23067132455999617,
 'H': 0.2476364956250818,
 'R': 0.28690542314091916,
 'rl': 0.48808056673088396}

### link_prediction_eval.sh

#### evaluate.py

In [150]:
from sklearn import linear_model
from keras.models import Sequential 
from keras.layers import Dense, Activation

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [151]:
def train(kb, kb_inv, name_path):
    train_pairs,train_labels = read_samples(task_path,'train.pairs')
    training_features = [[int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path] for pair in train_pairs]
    model = Sequential()
    input_dim = len(name_path)
    model.add(Dense(1, activation='sigmoid' ,input_dim=input_dim))
    model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(training_features, train_labels, nb_epoch=300, batch_size=128)
    return model

In [191]:
y_scores = [model.predict(np.reshape([int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path],[1,-1])) for pair in test_pairs]
ap_link = ap(y_scores,test_labels)
mAP = np.mean(ap_link)
print('mAP:',mAP)

mAP: 0.5511950207386302
